# 1. Data Processing

In [428]:
# import the libraries 
import numpy as np
import pandas as pd 
from time import time
from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp

In [408]:
# Read Data 
def read_data(file):
    with open(file, 'r') as file:
        # read first line 
        n, d, a, b = [int(x) for x in file.readline().split()] 

        # Matrix (n,d) full 0, if staff i rest day d(i) -> convert to 1 
        F = np.full((n, d), 0)  
        for staff in range(n):
            # read each line to end, [:-1] bcs end of each line is -1 
            temp = [int(x) for x in file.readline().split()[:-1]]  #check each line from 2 -> i+1
            for day in temp:
                F[staff, day-1] = 1 # day-1 bcs index of list
    return n, d, a, b, F

# Input 
n, d, a, b, F = read_data("data.txt")

In [409]:
print("Number of Staffs: {}\nNumber of Days: {}\nRange staffs of a shift: {} -> {}\nList rest day of staff i:\n{}".format(n,d,a,b,F))

Number of Staffs: 9
Number of Days: 5
Range staffs of a shift: 1 -> 3
List rest day of staff i:
[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]


In [410]:
# Check status to apply color for each type of status
def status_color(value):
  if value == "Rest": 
    color = 'Green'
  elif value == "Nigh":
    color = 'Red'
  else:
    color = 'White'
  return 'background-color: %s' % color

# 2. CSOP

## 2.1. Optimization

In [411]:
# Declare the Model 
model = cp_model.CpModel()

# Create the Variables
# X[staff, day, shift] = 1 if staff i work on shift k of day j 
# X[staff, day, shift] = 0, otw
X = {} 
for staff in range(n):              # check each staff 
    for day in range(d):            # check each staff
        for shift in range(1,5):    # check each shift
            X[staff, day, shift] = model.NewIntVar(0,1,"X[{},{},{}]".format(staff,day,shift))

In [412]:
# Each day, a staff can only work one shift at most
for staff in range(n):    
    for day in range(d):   
        if F[staff, day] == 0:
            if day == 0:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 1)
            # If you work the night shift the day before, you can rest the next day
            else:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) + X[staff, day - 1, 4] == 1)
        else: # F[staff, day] == 1
            model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 0)

In [413]:
# Each shift in each day has at least [a] staffs and at most [b] staffs
for day in range(d):               
    for shift in range(1,5):    
        model.Add(sum([X[staff, day, shift] for staff in range(n)]) >= a)
        model.Add(sum([X[staff, day, shift] for staff in range(n)]) <= b)

In [414]:
# F(i): list of staff rest days i
# The maximum number of night shifts assigned to a specific staff is the smallest

max_night_shift = model.NewIntVar(1, int(d/2) + 1, 'max_night_shift') # limit rest day = 1/2 all days
# for loop add constraint confirm sum of all night shift of staff <= max_night_shift
for staff in range(n):
    model.Add(sum([X[staff, day, 4] for day in range(d)]) - max_night_shift <= 0)

In [415]:
# Objective Function
model.Minimize(max_night_shift)

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)
if __name__ == '__main__':
  if status == cp_model.OPTIMAL:
    print("Minimize of max night shift:", solver.ObjectiveValue())
  else: 
    print("No optimal solution!")

Minimize of max night shift: 1.0


## 2.2. Visualization

In [416]:
# Matrix (n,d,s=5) full 0, if staff i works day j, shift k -> 1 add to matrix; 0 otw
# S is work calendar of each staff 
S = np.full((n, d, 5), 0)
for staff in range(n):
    for day in range(d):
        for shift in range(1, 5):
            S[staff, day, shift] = int(solver.Value(X[staff, day, shift])) # return {0;1}

# Label days & Shift to visualize
days = np.array([f"Day {day}" for day in range(1,d+1)])
shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])

# Flat S by axis 0, use sum to return matrix include sum staff for each shift day
staff_per_shift = np.sum(S, axis=0)
# Create pandas DF to visualize
df_staff_shift = pd.DataFrame(data=staff_per_shift[:, 1:].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
df.style.background_gradient(cmap='Pastel2')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,2,3,2,1
Noon,1,2,2,1,3
Afternoon,3,2,1,2,2
Night,2,1,2,1,1


In [417]:
col = np.array([f"Staff {staff}" for staff in range(1,n+1)])
row = days
details = np.full((d,n),"Rest")

for day in range(d):
  for staff in range(n):
    for shift in range(1,5):
      if S[staff, day, shift] == 1:
        details[day,staff] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details = pd.DataFrame(data = details, index = row, columns = col)  
pf_details.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Morn,Noon,Afte,Morn,Morn,Afte,Nigh,Nigh,Afte
Day 2,Nigh,Noon,Morn,Morn,Afte,Afte,Rest,Rest,Noon
Day 3,Rest,Morn,Noon,Morn,Nigh,Nigh,Morn,Afte,Noon
Day 4,Afte,Rest,Morn,Nigh,Rest,Rest,Noon,Afte,Morn
Day 5,Noon,Nigh,Noon,Rest,Noon,Afte,Morn,Rest,Afte


# 3. IP

## 3.1. Optimization

In [418]:
# Read data
N, D, a, b, F = read_data("data.txt")

In [419]:
solver = pywraplp.Solver('ROSTERING_MIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
INF = solver.infinity()

# DECISION VARIABLES
x = {}
for i in range(N):
    for j in range(D):
        for k in range(1, 5):
            x[i, j, k] = solver.IntVar(0, 1, f'x[{i}, {j}, {k}]')

z = solver.IntVar(0, D, 'z')  # z is an auxiliary variable to facilitate the objective function

In [420]:
# CONSTRAINTS
# Each employee works no more than one shift every day
for i in range(N):
    for j in range(D):
        if F[i][j] == 0:
            cstr = solver.Constraint(-INF, 1)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)
            if j != 0:
                cstr.SetCoefficient(x[i, j-1, 4], 1)

In [421]:
# Employees can have a day off after having a night shift on the previous day
for i in range(N):
    for j in range(D):
        if F[i][j] == 0:
            cstr = solver.Constraint(1, 1)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)
                if j != 0:
                    cstr.SetCoefficient(x[i, j-1, 4], 1)

In [422]:
# Employees will not work on their off days
for i in range(N):
    for j in range(D):
        if F[i][j] == 1:
            cstr = solver.Constraint(0, 0)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)

In [423]:
# Each shift have at least alpha and beta employees at most
for j in range(D):
    for k in range(1, 5):
        cstr = solver.Constraint(a, b)
        for i in range(N):
            cstr.SetCoefficient(x[i, j, k], 1)

In [424]:
# OBJECTIVE FUNCTION
for i in range(N):  # the maximum night shift of any employee is minimize
    obj = solver.Constraint(0, INF)
    for j in range(D):
        obj.SetCoefficient(x[i, j, 4], -1)
        obj.SetCoefficient(z, 1)

In [425]:
solver.Minimize(z)
status = solver.Solve()

if __name__ == '__main__':
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal value:', solver.Objective().Value())
        for i in range(N):
            for j in range(D):
                for k in range(1, 5):
                    if x[i, j, k].solution_value() > 0:
                        print(f'Employee {i+1} works on day {j+1}, at shift {k}')
    else:
        print("No optimal solution!")

Optimal value: 1.0
Employee 1 works on day 1, at shift 2
Employee 1 works on day 2, at shift 2
Employee 1 works on day 3, at shift 2
Employee 1 works on day 4, at shift 2
Employee 1 works on day 5, at shift 1
Employee 2 works on day 1, at shift 4
Employee 2 works on day 3, at shift 3
Employee 2 works on day 5, at shift 3
Employee 3 works on day 1, at shift 3
Employee 3 works on day 2, at shift 3
Employee 3 works on day 3, at shift 1
Employee 3 works on day 4, at shift 3
Employee 3 works on day 5, at shift 2
Employee 4 works on day 1, at shift 1
Employee 4 works on day 2, at shift 3
Employee 4 works on day 3, at shift 2
Employee 4 works on day 4, at shift 3
Employee 4 works on day 5, at shift 4
Employee 5 works on day 1, at shift 2
Employee 5 works on day 2, at shift 1
Employee 5 works on day 3, at shift 4
Employee 5 works on day 5, at shift 1
Employee 6 works on day 1, at shift 2
Employee 6 works on day 2, at shift 1
Employee 6 works on day 3, at shift 3
Employee 6 works on day 4, at s

## 3.2. Visualization

In [426]:
S = np.full((n, d, 5), 0)
for staff in range(N):
    for day in range(D):
        for shift in range(1, 5):
            # add value to matrix S full 0 
            S[staff, day, shift] = int(x[staff, day, shift].solution_value())

shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])
days = np.array([f"Day {day}" for day in range(1,d+1)])
day_shifts = np.sum(S, axis=0)
day_shifts_solution = pd.DataFrame(data=day_shifts[:, 1:].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
day_shifts_solution.style.background_gradient(cmap='Pastel1')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,2,2,1,2
Noon,3,2,3,2,2
Afternoon,2,3,2,3,3
Night,1,1,1,1,1


In [427]:
col = np.array([f"Staff {staff}" for staff in range(1,n+1)])
row = days
details_ip = np.full((d,n),"Rest")

for r in range(d):
  for c in range(n):
    for shift in range(1,5):
      if S[c,r,shift] == 1:
        details_ip[r,c] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details_ip = pd.DataFrame(data = details_ip, index = row, columns = col)
pf_details_ip.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Noon,Nigh,Afte,Morn,Noon,Noon,Morn,Afte,Morn
Day 2,Noon,Rest,Afte,Afte,Morn,Morn,Nigh,Afte,Noon
Day 3,Noon,Afte,Morn,Noon,Nigh,Afte,Rest,Morn,Noon
Day 4,Noon,Rest,Afte,Afte,Rest,Afte,Noon,Nigh,Morn
Day 5,Morn,Afte,Noon,Nigh,Morn,Afte,Noon,Rest,Afte


# 4. Heuristics

## 4.1. Optimization

In [446]:
def input(filename):
    with open(filename) as f:
        N, D, a, b = [int(x) for x in f.readline().split()]
        F = [[0 for _ in range(D+1)] for _ in range(N+1)]
        for i in range(N):
            d = [int(x) for x in f.readline().split()[:-1]]
            if d:
                F[i][d[0]-1] = 1
    F = np.array(F)
    return N, D, a, b, F


filename = 'data.txt'
N, D, a, b, F = input(filename)

In [447]:
def select(N, off_today, off_nextday, a, b):
    '''
    :param off_today: number of employees cannot work today
    :param off_nextday: number of employees cannot work on the next day
    :return: z = minimum value of the night shift of an employee
    :return: add = number of employees need to add to suffice the bound
    '''
    z, add = 0, 0
    upper_today = N - off_today - 4*a  # upper bound of the number of employees working today
    lower_today = N - off_today - 4*b

    if upper_today < a or lower_today > b:
        return -1
    else:
        z = max(lower_today, a)

    upper_nextday = N - off_nextday - z - 4*a
    lower_nextday = N - off_nextday - z - 4*b

    if lower_nextday > b:
        z += (lower_nextday - b) # remove redundant employees
    elif upper_nextday < a:
        add = a - upper_nextday  # add employees to suffice the bound
    else:
        add = 0

    if z > b or z < a or add > off_nextday:
        return -1
    else:
        return z, add

In [448]:
def heuristics(N, D, a, b, F):
    num_night = np.full(N, 0)  # number of night shifts of each employee
    global x

    for j in range(D):
        off_today = np.array(F[:, j][:N])
        off_nextday = np.array(F[:, j+1][:N])

        if j != 0:
            for i in range(N):
                if x[i, j-1, 3] == 1:  # if employee i worked at the night shift on the previous day, then rest today
                    off_today[i] = 1

        # Select the possible minimum number of night shift
        if select(N, sum(off_today), sum(off_nextday), a, b) is False:
            print('No optimal solution found.')
            return -1
        else:
            z, add = select(N, sum(off_today), sum(off_nextday), a, b)
        remain = z - add

        # Assign the employee with minimum number of night shift (and absent on the next day) to today's night shift
        emp_off_nextday = np.array([i for i in range(len(off_nextday)) if off_nextday[i] == 1])
        off_night_nextday = np.array([num_night[i] for i in emp_off_nextday])

        while add > 0:
            emp_index = np.argmin(off_night_nextday)
            x[emp_off_nextday[emp_index], j, 3] = 1
            num_night[emp_off_nextday[emp_index]] += 1  # add 1 employee to today's night shift
            off_today[emp_off_nextday[emp_index]] = 1  # avoid working more than one shift in a day
            add -= 1

        # Assign other employees to the night shift if needed (choose among idle employees for today)
        emp_work_today = np.array([i for i in range(len(off_today)) if off_today[i] != 1])
        work_night_today = np.array([num_night[i] for i in emp_work_today])

        while remain > 0:
            emp_index = np.argmin(work_night_today)
            x[emp_work_today[emp_index], j, 3] = 1
            num_night[emp_work_today[emp_index]] += 1
            off_today[emp_work_today[emp_index]] = 1
            remain -= 1

        # Assign other employees to other shifts of today
        i, k = 0, 0
        while i < N and k < 3:
            if off_today[i] == 0:
                x[i, j, k] = 1
                off_today[i] = 1  # avoid assigning the same employee in a day
                k = (k+1) % 3
            i += 1
    return max(num_night)

In [ ]:
if __name__ == '__main__':
    x = np.full((N, D, 4), 0)  # solution matrix

    start = time()
    res = heuristics(N, D, a, b, F)
    end = time()
    print('The optimal value is:', res)
    print('The optimal solution is:')

    for i in range(N):
        for j in range(D):
            for k in range(4):
                if x[i, j, k] == 1:
                    print(f'Employee {i+1}: works on day {j+1}, at shift {k+1}')

    # print(x)
    print('Total execution time:', end-start)

1

## 4.2. Visualization

In [465]:
X = np.full((n,d,4),0)
S = np.full((n, d, 5), 0)

for staff in range(N):
    for day in range(D):
        for shift in range(4):
            S[staff, day, shift] = int(x[staff, day, shift])

shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])
days = np.array([f"Day {day}" for day in range(1,d+1)])
day_shifts = np.sum(S, axis=0)
day_shifts_solution = pd.DataFrame(data=day_shifts[:, :4].T, index=shifts, columns=days)

In [466]:
day_shifts_solution.style.background_gradient(cmap='Pastel2')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,3,3,2,2
Noon,3,2,2,2,2
Afternoon,2,2,2,2,2
Night,1,1,1,1,1


In [467]:
col = np.array([f"Staff {staff}" for staff in range(1,n+1)])
row = days
details_heu = np.full((d,n),"Rest")

for r in range(d):
  for c in range(n):
    for shift in range(1,5):
      if S[c,r,shift] == 1:
        details_heu[r,c] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details_ip = pd.DataFrame(data = details_heu, index = row, columns = col)
pf_details_ip.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Afte,Rest,Morn,Noon,Rest,Morn,Noon,Rest,Morn
Day 2,Rest,Afte,Rest,Morn,Noon,Rest,Morn,Noon,Rest
Day 3,Rest,Rest,Afte,Morn,Noon,Rest,Morn,Noon,Rest
Day 4,Rest,Rest,Rest,Afte,Morn,Noon,Rest,Morn,Noon
Day 5,Rest,Morn,Noon,Rest,Afte,Rest,Morn,Rest,Noon
